In [4]:
# use the mxnet to do the test
import pandas as pd
import numpy as np

In [5]:
# import the files should be the same, copy from sklearn project
train_path='/home/ubuntu/learn-with-other-kaggle/data/train.csv'
test_path='/home/ubuntu/learn-with-other-kaggle/data/test.csv'

train_data=pd.read_csv(train_path, index_col = 'Id')
test_data=pd.read_csv(test_path, index_col = 'Id')
train_data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [6]:
#shuffle the data
data = train_data.values
np.random.shuffle(data)
columns_list = train_data.columns

shuffled_data = pd.DataFrame(data, columns=columns_list)


features_train = shuffled_data.drop('Cover_Type', axis=1)
label_train = shuffled_data['Cover_Type']
label_train = label_train.apply(lambda x:x-1)
#print(label_train.max())

In [7]:
#print(label_train.unique())

In [8]:
import mxnet as mx
from mxnet import autograd, nd, gluon, init
from mxnet.gluon import data as gdata, loss as gloss, nn

In [9]:
valid_rate = 0.1
valid_index = int(train_data.shape[0]*(1-valid_rate))

In [10]:
# need to convert the pd dataframe to ndarray will be used in mxnet

data_train_nd = nd.array(features_train.values[0:valid_index])
label_train_nd = nd.array(label_train.values[0:valid_index])

data_valid_nd = nd.array(features_train.values[valid_index:])
label_valid_nd = nd.array(label_train.values[valid_index:])


In [11]:
print(data_train_nd.shape)
print(data_valid_nd.shape)

(13608, 54)
(1512, 54)


In [41]:
def get_net(ctx):
    net = nn.Sequential()
    net.add(nn.Dense(100, activation='relu'))
   # net.add(nn.Dense(30, activation='relu'))
    net.add(nn.Dense(7))
    net.initialize(ctx=ctx,init=init.Xavier())
    return net

In [42]:
loss=gloss.SoftmaxCrossEntropyLoss()

In [43]:
def train(net,epochs, batch_size, lr, lr_period, lr_decay, train_data, label_data, wd, ctx,valid_data=None, valid_label=None):
    data_iter = gdata.DataLoader(gdata.ArrayDataset(train_data,label_data), batch_size, shuffle=True)
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate':lr,'wd':wd})
    for epoch in range(0,epochs):
        train_loss = 0
        for X,y in data_iter:
            with autograd.record():
                y_hat=net(X.as_in_context(ctx))
                l = loss(y_hat, y.as_in_context(ctx))
            l.backward()
            trainer.step(batch_size)
            train_loss += l.mean().asscalar()
        if epoch > 1 and epoch % lr_period ==0:
            #lr = trainer.learning_rate*lr_decay if trainer.learning_rate*lr_decay >0.00001 else 0.00001
            lr = trainer.learning_rate*lr_decay
            trainer.set_learning_rate(lr)

        if valid_data is not None:
            valid_loss = loss(net(valid_data.as_in_context(ctx)),valid_label.as_in_context(ctx)).mean().asscalar()
            if epoch%10 == 0:
                print("epoch %d, Train loss %f, learning rate %f, valid_error %f"%(epoch, train_loss/len(data_iter), 
                                                                               trainer.learning_rate, valid_loss))
        else:
            if epoch%10 == 0:
                print("epoch %d, Train loss %f, learning rate %f"%(epoch, train_loss/len(data_iter), 
                                                                               trainer.learning_rate))

In [45]:
net=get_net(mx.gpu())
train(net, 1000, 50, 0.005, 60, 0.5, data_train_nd, label_train_nd, 5e-4, mx.gpu(), data_valid_nd,label_valid_nd)

epoch 0, Train loss 30.276760, learning rate 0.005000, valid_error 29.319826
epoch 10, Train loss 2.970519, learning rate 0.005000, valid_error 2.766277
epoch 20, Train loss 0.929497, learning rate 0.005000, valid_error 0.887433
epoch 30, Train loss 0.912105, learning rate 0.005000, valid_error 0.985022
epoch 40, Train loss 0.956712, learning rate 0.005000, valid_error 0.926211
epoch 50, Train loss 0.998421, learning rate 0.005000, valid_error 0.970056
epoch 60, Train loss 1.055178, learning rate 0.002500, valid_error 1.022920
epoch 70, Train loss 0.990329, learning rate 0.002500, valid_error 0.964160
epoch 80, Train loss 0.972052, learning rate 0.002500, valid_error 0.988213
epoch 90, Train loss 0.976490, learning rate 0.002500, valid_error 1.054268
epoch 100, Train loss 0.961436, learning rate 0.002500, valid_error 0.996256
epoch 110, Train loss 0.950506, learning rate 0.002500, valid_error 0.930724
epoch 120, Train loss 0.956749, learning rate 0.001250, valid_error 0.954169
epoch 13

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/_ctypes/ndarray.py", line 51, in __del__
    check_call(_LIB.MXNDArrayFree(self.handle))
AttributeError: handle


epoch 160, Train loss 0.906649, learning rate 0.001250, valid_error 0.927737
epoch 170, Train loss 0.913813, learning rate 0.001250, valid_error 0.891523
epoch 180, Train loss 0.899743, learning rate 0.000625, valid_error 0.894926
epoch 190, Train loss 0.885808, learning rate 0.000625, valid_error 0.890265
epoch 200, Train loss 0.881673, learning rate 0.000625, valid_error 0.897281
epoch 210, Train loss 0.878856, learning rate 0.000625, valid_error 0.873423
epoch 220, Train loss 0.888113, learning rate 0.000625, valid_error 0.895761
epoch 230, Train loss 0.879319, learning rate 0.000625, valid_error 0.893391
epoch 240, Train loss 0.882475, learning rate 0.000313, valid_error 0.879950
epoch 250, Train loss 0.870654, learning rate 0.000313, valid_error 0.873726
epoch 260, Train loss 0.869188, learning rate 0.000313, valid_error 0.872551
epoch 270, Train loss 0.859370, learning rate 0.000313, valid_error 0.859524
epoch 280, Train loss 0.855381, learning rate 0.000313, valid_error 0.859365

In [46]:
output = net(nd.array(test_data).as_in_context(mx.gpu()))

In [ ]:
label = nd.argmax(output, axis=1)

In [ ]:
result = pd.concat([pd.Series(test_data.index), pd.DataFrame({'Cover_Type':label.asnumpy().astype(int)}).apply(lambda x:x+1)], axis=1)

In [ ]:
result.head()

In [ ]:
result.to_csv('mlp_submission.csv', index=False)